In [2]:
# Horse Image Classification with PyTorch
# =======================================

# Step 1: Import Required Libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Step 2: Define a Custom Dataset Class
class HorseDataset(Dataset):
    def __init__(self, metadata_df, transform=None):
        self.metadata_df = metadata_df
        self.transform = transform

    def __len__(self):
        return len(self.metadata_df)

    def __getitem__(self, idx):
        row = self.metadata_df.iloc[idx]
        image_path = row['image_path']
        label = row['horse_id'] - 1  # Subtract 1 to convert to 0-based index

        # Load and preprocess the image
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
# Step 3: Prepare the Data
def load_and_prepare_data(metadata_df, test_size=0.2, random_state=42):
    # Define transformations for images
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    # Split metadata into training and testing sets
    train_df, test_df = train_test_split(metadata_df, test_size=test_size, random_state=random_state, stratify=metadata_df['horse_id'])

    # Create PyTorch datasets
    train_dataset = HorseDataset(train_df, transform=transform)
    test_dataset = HorseDataset(test_df, transform=transform)

    return train_dataset, test_dataset

In [5]:
# Step 4: Define the CNN Model
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [6]:
# Step 5: Define the Training Loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

In [8]:

# 2. Prepare data for model training:
# Base path to the database
base_dir = '../data/THoDBRL2015'

metadata = []

# Consolidate images into a training directory
output_train_dir = os.path.join(base_dir, 'training_data')

os.makedirs(output_train_dir, exist_ok=True)

parts = [
    'Part1', 
    # 'Part2', 
    # 'Part3', 
    # 'Part4', 
    # 'Part5'
]

# Iterate through each Part folder
for part in parts:
    videos_dir = os.path.join(base_dir, part, 'videos')

    for horse_id_folder in os.listdir(videos_dir):
        horse_path = os.path.join(videos_dir, horse_id_folder)

        if os.path.isdir(horse_path):  # Ensure it's a directory
            # Find all stills folders (e.g., images, images1, images2, etc.)
            for stills_folder in os.listdir(horse_path):
                stills_path = os.path.join(horse_path, stills_folder)

                if os.path.isdir(stills_path) and stills_folder.startswith('images'):  # Check for folders named 'images*'
                    target_dir = os.path.join(output_train_dir, f'horse_{horse_id_folder}')

                    os.makedirs(target_dir, exist_ok=True)
                    
                    # Copy all image files from the stills folder to the target directory
                    for img_file in os.listdir(stills_path):
                        img_path = os.path.join(stills_path, img_file)

                        if img_file.endswith(('.jpg', '.jpeg', '.png')):  # Ensure it's an image file
                            metadata.append({
                                'horse_id': horse_id_folder,
                                'image_path': img_path,
                                'width': Image.open(img_path).size[0],
                                'height': Image.open(img_path).size[1],
                            })
                        
                            # shutil.copy(img_path, target_dir)

                    # print(f"Copied images from {stills_path} to {target_dir}")

# Create DataFrame with specified dtypes
metadata_df = pd.DataFrame(metadata, dtype='object').astype({
    'horse_id': 'int64',          # Assuming it's an integer
    'image_path': 'string',
    'width': 'int64',
    'height': 'int64',
})




In [9]:
# Step 6: Load Metadata and Prepare the Dataset
# Replace with your actual metadata DataFrame
# metadata_df = pd.DataFrame({
#     'image_path': ['path_to_image_1.jpg', 'path_to_image_2.jpg'],  # Replace with actual paths
#     'horse_id': [0, 1]
# })

# Prepare training and testing datasets
train_dataset, test_dataset = load_and_prepare_data(metadata_df)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [14]:
# Step 7: Initialize the Model, Loss Function, and Optimizer
num_classes = metadata_df['horse_id'].nunique()
print(metadata_df['horse_id'].unique())
model = CNNModel(num_classes)

# Use Cross-Entropy Loss for classification
criterion = nn.CrossEntropyLoss()

# Use Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

[ 9  7  6  1 10  8  4  3  2  5]


In [12]:
# Step 8: Train the Model
# Set the device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=2, device=device)

IndexError: Target 10 is out of bounds.